In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json

In [2]:
indices = ['hnsw', 'vamana', 'hcnng', 'nsg']
colors = ['tomato', 'deepskyblue', 'forestgreen', 'PeachPuff']
distr_list = [
  'uniform-0.0-1.0',
  'normal-0.0-1.0',
  'poisson-100.0'
]
distr_map = {
  'uniform-0.0-1.0': r"U(0,1)",
  'normal-0.0-1.0': r"N(0,1)",
  'poisson-100.0': r"P(100)"
}

In [ ]:
from function import pareto_frontier
from matplotlib.ticker import ScalarFormatter

nrows = 1
ncols = len(distr_list)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(9, 1.5))

# for i in range(nrows):
axes[0].set_ylabel('QPS', fontsize=14)
# for i in range(num_datasets):
for j in range(len(distr_list)):
  distr = distr_list[j]
  ds = 'sift-128-euclidean'
  for k in range(len(indices)):
    index = indices[k]
    recall_limit = 0.5
    recall_gap = 0.1
    with open(f"perf/overall/pf.{index}.{ds}.{distr}.{distr}.k10.json", 'r') as file:
      baseline = pareto_frontier(json.load(file), recall_limit)
    with open(f"perf/overall/qt.{index}.{ds}.{distr}.{distr}.k10.json", 'r') as file:
      method = pareto_frontier(json.load(file), recall_limit)
    # baseline = [(item['qps'], item['recall']) for item in baseline if item['recall'] > recall_limit]
    # method = [(item['qps'], item['recall']) for item in method if item['recall'] > recall_limit]
    axes[j].plot([item['recall'] for item in baseline], 
                                [item['qps'] for item in baseline], marker='s', 
                                  color=colors[k], linewidth=1.2, linestyle='dashed', markersize=3)
    axes[j].plot([item['recall'] for item in method], 
                                [item['qps'] for item in method], marker='o', 
                                  color=colors[k], linewidth=1.2, markersize=3)
  # if i == nrows - 1:
  axes[j].set_xlabel(f'Recall@10', fontsize=14)
  axes[j].set_xticks(np.arange(recall_limit, 1.001, recall_gap))
  # axes[j].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
  axes[j].set_yscale("log", base=10)
  axes[j].grid(True)
  axes[j].set_title(f"SIFT1M ({distr_map[distr]})",fontsize=14)

leg = list()
for idx in indices:
  if idx != 'vamana':
    leg += [idx.upper(), 'Hi-PNG-' + (idx).upper()]
  else:
    leg += ['Vamana', ('Hi-PNG-Vamana')]

fig.subplots_adjust(wspace=0.5)
# fig.legend(leg, loc='center', bbox_to_anchor=(0.51, 1.35), ncol=len(indices), frameon=False, fontsize=14)
fig.savefig(f'figure/factor/distribution.pdf', dpi=300, bbox_inches='tight', pad_inches=0)
fig.show()